# Estimating rainfall from radar data

## Radar data to rain intensity

The data we have available is a collection of 1024x1024 PNG images where the data at each pixel is related to the reflected power coming from the location represented by the pixel back to the radar. The actual position of each pixel can be inferred by using 'radarfootprint.tif' a GeoTiff image centered on the radar location.
The PNG image contains two bands, band0 with the actual signal and band1 with a mask array that is 0 where the signal should be zero and 255 everywhere else. The discretized signal, $D$, is bound to the related effective reflected power $P'$

$$
D= (100 + P') * 2.55
$$

Where

$$
P' = P +  20 \log(r)
$$

$P$ is the actual reflected at the radar and $P'$ includes a distance correction term.

On the other hand, the reflected power $P$ is connected to $Z$, the reflectivity, a quantity that can be measured by the radar, and it is defined as 

$$
Z = \sum_d N_d  d^6 \delta d
$$

Where $N_d\delta d$ is the number of drops per cubic meter of diameter between $d$ and $d + \delta d$ mm.
The relation between $Z$ and $P$, assuming that the reflection happens at $r$ km away is given by

$$
10 \log(Z) = P + 91.4 + 20 \log(r)
$$

Where all the specific constants that characterize this specific radar have been subsumed in global constants.
Moreover, the (Marshall-Palmer distribution of raindrops) [ https://doi.org/10.1016/0960-1686(93)90066-8 , see also http://www.sjsu.edu/faculty/watkins/raindrop.htm ] relates $Z$ to the related rainfall $R$ by

$$
Z = a R^b
$$

where, in our case,  $a=300$, $b=1.5$.

As a result, we can now invert the equations above to obtain a formula that relates the reflectivity to $D$.

$$
Z = 10^\frac{D/2.55 - 8.6}{10}
$$

In [ ]:
from tdm.radar import utils
from datetime import datetime, timedelta
import numpy as np
import imageio
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def get_image_data(path):
    im = imageio.imread(path)
    return im[:,:,0], im[:,:,3] == 255 # NOTE the mask is on channel 3!!!

def aggregate_rainfall(image_tuples):
    precs = np.dstack([utils.estimate_rainfall(*get_image_data(_[1])) for _ in image_tuples])
    return np.mean(precs, axis=2)

In [ ]:
root = './data/radarsample/cag01est2400'
template = './data/radarsample/radarfootprint.tif'
sample_image_dt = '2018-05-01_23:01:05'
after = datetime(2018, 5, 1, 23, 0, 0)
before = datetime(2018, 5, 1, 23, 40, 0)
delta = timedelta(minutes=10)

In [ ]:
signal, mask = get_image_data("data/radarsample/cag01est2400/%s.png" % sample_image_dt)

In [ ]:
plt.figure(dpi=144)
plt.contourf(signal, 10)
plt.gca().set_aspect("equal")

In [ ]:
levels = np.arange(0, 3, 0.1)

In [ ]:
rain = utils.estimate_rainfall(signal, mask)

In [ ]:
plt.figure(dpi=144)
c = plt.contourf(rain, levels=levels) # mm/min ??
cbar = plt.colorbar(c)
cbar.ax.set_ylabel('rainfall (mm/min)')
plt.gca().set_aspect("equal")

In [ ]:
fname = "%s.3003.tif" % sample_image_dt
metadata = {'TIFFTAG_DATETIME': sample_image_dt}
ga = utils.GeoAdapter(template)
ga.save_as_gtiff(fname, rain, metadata)

In [ ]:
image_groups = utils.get_grouped_raw_radar_images(root, delta, after, before)

In [ ]:
for g, ims in image_groups:
    rain = aggregate_rainfall(ims)
    plt.figure(dpi=144)
    c = plt.contourf(rain, levels=levels)
    plt.title("%d min from %s" % (delta.total_seconds()/60, g))
    cbar = plt.colorbar(c)
    cbar.ax.set_ylabel('rainfall (mm/min)')
    plt.gca().set_aspect("equal")